In [1]:
#Programa donde se agregan primero la columna de ID al df de lugares que ya tiene los estratos
#luego se agregan columnas de numero de opiniones,numero de fotos, calificacion, numero de calificadores, precios
import pandas as pd
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import requests
from time import time

In [2]:
#Lectura de el dataframe de lugares de bogota con estrato e id (esta ya fue limpiado)
path='./LugaresEstratoL-V2.csv'
dfLugaresL=pd.read_csv(path)
dfLugaresL=dfLugaresL.drop(columns='Unnamed: 0')

In [47]:
dfLugaresL

,name,categories,lat,lng,id,NumPunto,Estrato
0,El Fogón Del Llano,Cajun / Creole Restaurant,4.468930,-74.127996,4f4a782de4b0cccedd61818b,134,2
1,Plaza de Usme,Plaza,4.469970,-74.126363,4f4a3862e4b0eddb52dd97bb,135,2
2,Juan Valdez,Café,4.477763,-74.122131,537260f7498e7c622a9bf03a,245,1
3,agroparque los soches,Field,4.483056,-74.091950,51ebe330498eecb9a833ac50,367,1
4,Lacteos Santa Monica,Health Food Store,4.490626,-74.148808,5bd1f68e12c8f0002cc8b39c,445,2
...,...,...,...,...,...,...,...
7483,TÍPICA sopas y picadas,Steakhouse,4.815070,-74.031742,53d41873498e67ffa980e88d,14903,6
7484,Club Chalaneria Guadalupe,Stables,4.817814,-74.069090,563e6a79498e41debf3b8dab,14926,6
7485,Al Limite Paintball,Paintball Field,4.818875,-74.032603,4ca77d9ba6e08cfa63508594,14944,6
7486,Refugio Hípica - Club Guaymaral,Stables,4.820947,-74.039281,503913f2e4b0cc82fd815e42,14978,6


In [48]:
#Credenciales para hacer solicitudes a FourSquare

#Mis credenciales (David Mora damorom@gmail)
#CLIENT_ID = 'HW5FTE3WWTW152NHJSCCIKW34ZPNZPOMSFAXOXNWABWRVNSB' # your Foursquare ID
#CLIENT_SECRET = 'CRAGUXJAIPNIQW4I2X3LTHUJ2KWIEK5TUHYZM5ETSRJOKOVH' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version


#Mis credenciales 2(David Mora UNAL)
#CLIENT_ID = '41K3O1CU3LV51Z135WACNGEMXZ5XHNEKAIEBEBJ52NP5TPFO' # your Foursquare ID
#CLIENT_SECRET = 'JZXDMR3L5LNPXZ4GW0IKHXDFMQEIOJ4JVC2C5R0NDDJZR2FW' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Mis credenciales 3(David Mora USTA)
#CLIENT_ID = 'XJK1HQGKQRDZDXIECU5EZPTIANQIIOIQRTMBMBGBKBSO5ZB5' # your Foursquare ID
#CLIENT_SECRET = 'DN4BNBAHXTHE0TQG4PV2MOTBMOT2PXTCW3HEGZX055BW0X4V' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Mis credenciales 4(David Mora HOTMAIL)
#CLIENT_ID = 'X3FG44YQT1K0V0KNAYMH21USAFEOV5AP5GZ4FEWITUL1PQXP' # your Foursquare ID
#CLIENT_SECRET = 'YZUYRKW1QU5PKU02KEJ5P0NIILYOCGGZW24X5M0EXI4BLQTZ' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Credenciales de linda (Alejandra Paez)
#CLIENT_ID = 'JAX1FV1DBLVZNQAHRJVEQLMRO55RENJGNHO0NRI2D1LC2LJQ' # your Foursquare ID
#CLIENT_SECRET = 'NOYNK5VDCYGN0QGRJEZP0DTSDDSD0H2AVTXFW00KRDUSIFD4' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Credenciales de linda 2(Alejandra Paez)
CLIENT_ID = 'QX2TLLFVNV0EXREI0Z1LIVLWZU3ZVWBBQOVI5M0H5EHUYF5V' # your Foursquare ID
CLIENT_SECRET = 'FICXOJSWZX04GMMA5URZJMH0JPE15BRUWZTMOJYJ2OMGOTYT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#Credenciales de linda 3(Alejandra Paez)
#CLIENT_ID = '1BYV5UHE4JLYXVBDPIJCOO15DONN1RBDAVRX43XAVRHBG40X' # your Foursquare ID
#CLIENT_SECRET = '2CIXX42KDRAE3GEN3HDWK3D33NP4EFX14205OBGHXYAQOCOE' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Credenciales de linda 4(Alejandra Paez)
#CLIENT_ID = 'UZXQL5EAXHLGLWNKFOZDA3AHAVYRTYTMPUXESSMTBO12UGMH' # your Foursquare ID
#CLIENT_SECRET = '0BZLW3XV42OA05C3OB32H4XUVUBVLGLKZU4OPAJSMOMDWHSA' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#Credenciales de papa (Jairo Mora)
#CLIENT_ID = 'TS0FSEPKAPDDVEEQIKLTVL1TFQLJY2FTKORIMB1U0UDARQVC' # your Foursquare ID
#CLIENT_SECRET = 'ENJ1RZ5JBZJTFQSIBJP5NY5EUYMT5OVFN2Q1055TWOCKTDOX' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version


#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [49]:
#funcion que solicita la informacion de un lugar y la ordena en una lista. De entrada tiene el id del
#lugar y entrega la informacion en este orden rating, numero de calificadores del rating, cantidad de 
#fotos, cantidad de tips
def InfoLugar(id):
    urllugar='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(id,CLIENT_ID, CLIENT_SECRET,VERSION)
    jsonlugar = requests.get(urllugar).json()
    Info=[]
    
    #agrega numero de calificadores del rating
    try:
        Info.append(jsonlugar['response']['venue']['ratingSignals']) 
    except:
        Info.append('SC')
        #print('Sin calificadores')
    
    #agrega rating
    try:
        Info.append(jsonlugar['response']['venue']['rating']) 
    except:
        Info.append('NR')
        #print('Sin rating')
    
    #agrega cantidad de fotos
    Info.append(jsonlugar['response']['venue']['photos']['count']) 
    
    #agrega cantidad de opiniones
    Info.append(jsonlugar['response']['venue']['stats']['tipCount'])
    
    return Info
    

In [58]:
#Programa que hace las peticionas de info de cada lugar y las agrega al dataframe, de entrada tiene 
#el dataframe con una columna titulala 'id' con los id de cada lugar y entrega el dataframe con la 
#informacion, como se pueden hacer un numero reducido de peticiones por dia se haran varios dataframes
#y luego se uniran

###Primer dia de peticiones:###
#temp0 va desde el indice 0 a 499 peticion hecha desde DM
#temp1 va desde el indice 500 a 545 peticion hecha desde JM
#temp2 va desde el indice 546 a 595 peticion hecha desde AP1
#temp3 va desde el indice 596 a 645 peticion hecha desde AP2
#temp4 va desde el indice 646 a 695 peticion hecha desde AP3
#temp5 va desde el indice 696 a 745 peticion hecha desde AP4
#temp6 va desde el indice 746 a 795 peticion hecha desde DM3
#temp7 va desde el indice 796 a 845 peticion hecha desde DM2

###Segundo dia de peticiones:###
#temp8 va desde el indice 846 a 895 peticion hecha desde JM
#temp9 va desde el indice 896 a 945 peticion hecha desde AP1
#temp10 va desde el indice 946 a 995 peticion hecha desde AP2
#temp11 va desde el indice 996 a 1045 peticion hecha desde AP3
#temp12 va desde el indice 1046 a 1095 peticion hecha desde AP4
#temp13 va desde el indice 1096 a 1595 peticion hecha desde DM1
#temp14 va desde el indice 1596 a 1646 peticion hecha desde DM2 #No es error, extrañamente acepto una peticion de mas
#temp15 va desde el indice 1647 a 1696 peticion hecha desde DM3
#temp16 va desde el indice 1697 a 2146 peticion hecha desde JM #ya permitio hacer 500
#temp17 va desde el indice 2147 a 2148 peticion hecha desde DM3 #Si, solo dos 

###Tercer dia de peticiones:###
#temp18 va desde el indice 2149 a 2648 peticion hecha desde DM1 #+500
#temp19 va desde el indice 2649 a 3148 peticion hecha desde DM2 #ya permitio hacer 500
#temp20 va desde el indice 3149 a 3648 peticion hecha desde DM3 #ya permitio hacer 500
#temp21 va desde el indice 3648 a 3698 peticion hecha desde DM4
#temp22 va desde el indice 3699 a 3748 peticion hecha desde AP1
#temp23 va desde el indice 3749 a 3798 peticion hecha desde AP2
#temp24 va desde el indice 3799 a 4298 peticion hecha desde AP3 #ya permitio hacer 500
#temp25 va desde el indice 4299 a 4348 peticion hecha desde AP4
#temp26 va desde el indice 4349 a 4848 peticion hecha desde JM #+500

###Cuarto dia de peticiones:###
#temp27 va desde el indice 4849 a 5348 peticion hecha desde DM1
#temp28 va desde el indice 5349 a 5848 peticion hecha desde DM2
#temp29 va desde el indice 5849 a 6348 peticion hecha desde DM3
#temp30 va desde el indice 6349 a 6398 peticion hecha desde DM4
#temp31 va desde el indice 6399 a 6898 peticion hecha desde JM
#temp32 va desde el indice 6899 a 6948 peticion hecha desde AP4
#temp33 va desde el indice 6949 a 7448 peticion hecha desde AP3
#temp34 va desde el indice 7449 a 7486 peticion hecha desde AP2
#temp35 va desde el indice 7487 a 7487 peticion hecha desde AP2

temp35=pd.DataFrame(columns=['CRat','Rat','CFot','COpi'])
I=len(dfLugaresL)-1
start=time()

for i in range(7487,I+1):
    print('Lugar',i,'de',I)
    #print('tiempo: ',time()-start)
    #start=time()
    #print(dfLugaresL.loc[i]['id'])
    info=InfoLugar(dfLugaresL.loc[i]['id'])
    temp35=temp35.append(pd.Series(info,index=['CRat','Rat','CFot','COpi']),ignore_index=True)
#dfLugaresL=pd.concat([dfLugaresL,temp],axis=1)
         

Lugar 7487 de 7487


In [59]:
temp35

,CRat,Rat,CFot,COpi
0,55.0,8.0,36.0,8.0


In [60]:
temp35.to_csv('temp35.csv')

In [134]:
#######Ya no mas solicitudes a FourSquare!!!!!

#Finalmente se pudieron obtener todos los valores de calificacion de los lugares, ahora deben unirse al DF de luhares de bogota
#primero se deben leer todos los dataframes temporales
tempsList=[]
for i in range(36):
    vardf='temp'+str(i) 
    path='./'+vardf+'.csv'
    vars()[vardf]=pd.read_csv(path)
    tempsList.append(vars()[vardf])

In [139]:
#Concatenamos todos los temporales
dfCalLugares=pd.concat(tempsList, ignore_index=True)
dfCalLugares=dfCalLugares.drop(columns='Unnamed: 0')
dfCalLugares

,CRat,Rat,CFot,COpi
0,SC,NR,0.0,0.0
1,SC,NR,12.0,0.0
2,SC,NR,0.0,0.0
3,SC,NR,3.0,0.0
4,SC,NR,0.0,0.0
...,...,...,...,...
7483,12.0,6.4,13.0,5.0
7484,SC,NR,0.0,0.0
7485,37.0,8.2,51.0,13.0
7486,SC,NR,11.0,1.0


In [140]:
#ahora se unira por filas el dataframe de lugares con el de calificaciones
dfLugaresComplete = pd.concat([dfLugaresL, dfCalLugares], axis=1)
dfLugaresComplete

,name,categories,lat,lng,id,NumPunto,Estrato,CRat,Rat,CFot,COpi
0,El Fogón Del Llano,Cajun / Creole Restaurant,4.468930,-74.127996,4f4a782de4b0cccedd61818b,134,2,SC,NR,0.0,0.0
1,Plaza de Usme,Plaza,4.469970,-74.126363,4f4a3862e4b0eddb52dd97bb,135,2,SC,NR,12.0,0.0
2,Juan Valdez,Café,4.477763,-74.122131,537260f7498e7c622a9bf03a,245,1,SC,NR,0.0,0.0
3,agroparque los soches,Field,4.483056,-74.091950,51ebe330498eecb9a833ac50,367,1,SC,NR,3.0,0.0
4,Lacteos Santa Monica,Health Food Store,4.490626,-74.148808,5bd1f68e12c8f0002cc8b39c,445,2,SC,NR,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
7483,TÍPICA sopas y picadas,Steakhouse,4.815070,-74.031742,53d41873498e67ffa980e88d,14903,6,12.0,6.4,13.0,5.0
7484,Club Chalaneria Guadalupe,Stables,4.817814,-74.069090,563e6a79498e41debf3b8dab,14926,6,SC,NR,0.0,0.0
7485,Al Limite Paintball,Paintball Field,4.818875,-74.032603,4ca77d9ba6e08cfa63508594,14944,6,37.0,8.2,51.0,13.0
7486,Refugio Hípica - Club Guaymaral,Stables,4.820947,-74.039281,503913f2e4b0cc82fd815e42,14978,6,SC,NR,11.0,1.0


In [141]:
dfLugaresComplete.to_csv('LugaresBogotaCompleto.csv')